<a href="https://colab.research.google.com/github/OlyKoek/Udemy-AIperfectMaster-colabo/blob/main/embeding_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# word2vec and doc2vec学習

## Sentence-Transformer体験(doc2vec系)

In [ ]:
# --- Colab 初期設定 ---
!pip install -q sentence-transformers faiss-cpu

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [ ]:
# モデル選択（日本語含む多言語対応）
# SentenceTransformer モデルをロードします。
# "paraphrase-multilingual-MiniLM-L12-v2" は、多くの言語に対応した比較的小さなモデルです。
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [ ]:
# テスト語彙
# 埋め込みベクトルを取得したい単語のリストを定義します。
# words = ["猫", "犬", "人間", "先生", "学生", "こころ", "恋", "友達"]
words = ["王", "エリザベス","男","女","女性","おじいさん","おばあさん","妻","夫"]

# 埋め込みを取得
# 定義した単語リストの埋め込みベクトルを取得します。
# convert_to_numpy=True で結果を NumPy 配列として取得します。
# normalize_embeddings=True で埋め込みベクトルを正規化し、コサイン類似度の計算を容易にします。
embeddings = model.encode(words, convert_to_numpy=True, normalize_embeddings=True)
print(f"単語数: {len(words)}")
print(f"埋め込み数: {len(embeddings[0])}") # 各単語の埋め込みベクトルの次元数

In [ ]:
# コサイン類似度計算関数
# 指定した単語に最も類似する単語を見つける関数を定義します。
def most_similar(word, topn=10):
    # 指定した単語がリストに存在するか確認します。
    if word not in words:
        print(f"{word} はリストに存在しません。")
        return
    # 指定した単語のインデックスを取得します。
    idx = words.index(word)
    # 指定した単語の埋め込みベクトルと他の全ての単語の埋め込みベクトルとのコサイン類似度を計算します。
    sims = util.cos_sim(embeddings[idx], embeddings)[0].cpu().numpy()
    # 類似度が高い順にインデックスをソートします。
    sorted_idx = np.argsort(-sims)
    # 結果を表示します。
    print(f"\n🔎 「{word}」に類似する単語:")
    # 類似度が高い順に上位 topn 個の単語と類似度を表示します。（最初の要素は自分自身なのでスキップします）
    for i in sorted_idx[1:topn+1]:
        print(f"  {words[i]} : 類似度 {sims[i]:.3f}")

# 試してみる
# most_similar("吾輩") # コメントアウトされています
most_similar("王")
# most_similar("先生") # コメントアウトされています

### 少し長い文章同士の比較

In [ ]:
# 比較したい日本語の文章リストを定義します。
sentences = [
    "猫は可愛い動物です。",
    "犬は忠実なペットです。",
    "動物園には様々な動物がいます。",
    "今日の天気は晴れです。",
    "この本はとても面白いです。",
    "あの猫はとても可愛いです。",
    "彼の犬はとても賢いです。",
]

# 定義した文章リストに対して、SentenceTransformer モデルを使用して埋め込みベクトルを計算します。
# convert_to_numpy=True で結果を NumPy 配列として取得し、normalize_embeddings=True で正規化します。
sentence_embeddings = model.encode(sentences, convert_to_numpy=True, normalize_embeddings=True)

print(f"文章数: {len(sentences)}")
print(f"文章埋め込み数: {len(sentence_embeddings[0])}") # 各文章の埋め込みベクトルの次元数

# 計算された文章埋め込みベクトル間のコサイン類似度を計算します。
# util.cos_sim 関数は、2つの埋め込みセット間のコサイン類似度行列を計算します。
similarity_matrix = util.cos_sim(sentence_embeddings, sentence_embeddings).cpu().numpy()

# 計算された類似度行列を表示します。
# DataFrame を使用すると、どの文章ペアの類似度が高いかが見やすくなります。
import pandas as pd

similarity_df = pd.DataFrame(similarity_matrix, index=sentences, columns=sentences)

print("\n文章間のコサイン類似度行列:")
display(similarity_df)

## FastT体験 (word2vec系)

In [ ]:
# bpemb ライブラリをインストールします。これは FastText の事前学習済みモデルを簡単に利用するためのライブラリです。
!pip install bpemb
# BPEmb クラスをインポートします。
from bpemb import BPEmb

In [ ]:
# 日本語の BPEmb モデルをロードします。
# lang="ja" で日本語を指定し、dim=300 で埋め込みベクトルの次元数を300に指定します。
bpemb_ja = BPEmb(lang="ja", dim=300)
# "猫" という単語の埋め込みベクトルを取得して表示します。
bpemb_ja.embed("猫")

In [ ]:
# "吾輩は" という単語列に最も類似する単語を取得します。
# BPEmb は単語をサブワードに分割して処理します。
bpemb_ja.most_similar(["吾","輩","は"])

In [ ]:
# 単語アナロジーを計算します。
# positive には類似度を加えたい単語、negative には類似度を減らしたい単語を指定します。
# この例では、「王」と「男」のベクトルを足し、「強」のベクトルを引いた結果に最も近い単語を検索しています。
bpemb_ja.most_similar(positive=["王","男"], negative=["強"])

## Fasttext でoos (out-of-vocabulary) 単語を扱う例を追加する

### Subtask:
FastText が未知語（OOV）をどのように扱えるかを示す新しいコードセルとその説明を追加します。


**Reasoning**:
The previous code block failed because the `bpemb_ja` object does not have a `vector()` method. The correct way to get the combined vector for a word (including OOV words) is to get the subword embeddings using `embed()` and then sum or average them. Let's fix the code to reflect this.



In [ ]:
# OOV 単語を定義します。この単語は学習データに含まれていない可能性が高いです。
oov_word = "ピヨピヨ語" # 例：存在しない可能性の高い架空の単語

# 定義した未知語に対して、BPEmb モデルを使用して埋め込みベクトルを取得します。
# FastText のサブワードベースのアプローチにより、未知語でも埋め込みが生成されます。
# bpemb_ja.embed() はサブワードごとの埋め込みのリストを返します。
oov_embeddings = bpemb_ja.embed(oov_word)

print(f"未知語「{oov_word}」のサブワード埋め込み（次元数: {len(oov_embeddings[0])}）:")
# 埋め込みベクトルを表示します。
display(oov_embeddings)

# 未知語の合計ベクトルは、その単語を構成するサブワードの埋め込みを合計することで得られます。
# この合計ベクトルが、多くの場合、単語全体の埋め込みとして使用されます。
oov_vector = np.sum(oov_embeddings, axis=0)
print(f"\n未知語「{oov_word}」の合計ベクトル（単語埋め込みとして使用可能）:")
display(oov_vector)

# さらに、未知語がどのようにサブワードに分割されたかを確認することもできます。
subwords = bpemb_ja.encode(oov_word)
print(f"\n未知語「{oov_word}」のサブワード分割:")
display(subwords)

bpemb_ja.most_similar(positive=[oov_vector])